##Imports

In [22]:
import re
import pandas as pd
import json

# Extração de Dados

In [48]:
def df_tweets_candidatos(json_filename):
    df = pd.read_json(json_filename).drop(columns = ['replies'])
    return df


def df_tweets_respostas(json_filename):
    df = pd.read_json(json_filename)[['replies']]
    ds = []
    for replies in df['replies'].to_list():
        for reply in replies:
            ds.append(reply)

    reply_df = pd.DataFrame (ds).drop(columns = ['attachments'])
    return reply_df

def df_tweets_cadidatos_respostas(json_filename):
    df_tweets_cand = df_tweets_candidatos(json_filename)
    df_tweets_reps = df_tweets_respostas(json_filename)

    return (df_tweets_cand, df_tweets_reps)

In [ ]:
def normalize_df(df):
  #word_regex = r"[-'a-zA-ZÀ-ÖØ-öø-ÿ]+" #para capturarmos palavras dentro do tweet
  #df['text'] = re.sub(r'^https?:\/\/.*[\r\n]*', '', df['text'], flags=re.MULTILINE)  #para remover os links do campo de texto do tweet

# Pré-Processamento

# Modelagem de Tópicos

*texto em itálico*# Análise de Sentimentos

# Similaridade de Textos